In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader


import numpy as np
import matplotlib.pyplot as plt

## Q3 dataloader

the following class reads the data for Q3 and creates a torch dataset object for it. With this, you can easily 
use a dataloader to train your model. 

Make sure that the file "hw2_Q3_data.npz" is located properly (in this example, it should be in the same folder as this notebook.

 



In [ ]:
class Q3Data(Dataset):
    def __init__(self,trn_tst = 0):
        data = np.load('./data/hw2_Q4_data.npz')
        if trn_tst == 0:
            #trainloader
            self.images = data['arr_0'].T
            self.labels = data['arr_1']
        else:
            #testloader
            self.images = data['arr_2'].T
            self.labels = data['arr_3']
            
        self.images = np.float32(self.images)/255.0


    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
   
        sample = self.images[idx,:]
        labels = self.labels[idx]
        return sample, labels

Here is an example of how you can create a dataloader for the Q3 data

In [ ]:
trainset =   Q3Data(trn_tst=0)  
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100,
                                         shuffle=True)


testset =   Q3Data(trn_tst=1)  
testloader = torch.utils.data.DataLoader(testset, batch_size=100,
                                         shuffle=False)